In [5]:
import h5py
from pathlib import Path
import numpy as np

In [6]:
# Filenames
fname1 = '/Users/lotzegud/P08/broken/h2o_2024_10_16_01116.nxs'
fname2 = '/Users/lotzegud/P08/broken/nai_250mm_02310.nxs'
fname3 = '/Users/lotzegud/P08/broken/nai_250mm_02311.nxs'
fname4= '/Users/lotzegud/P08/test_folder2/h2o_2024_10_16_01043.nxs'


#for fname in [fname1, fname2, fname3]:    
for fname in [fname4]:    
    with h5py.File(fname, 'r') as f:
        # Access the dataset
        dataset = f['/scan/instrument/lisa_sample/lisa_sample_stage/sz']
        data = dataset[...]  # Load the actual data
        print('Data size ',{data.size}, 'data shape ', data.shape)
        print(f"Dataset type: {type(dataset)}")  # This prints the type of the h5py dataset
        print(f"Data type: {data.dtype}")  # This prints the numpy dtype of the dataset's data
        print(f"Attributes: {dataset.attrs}")  # Attributes of the dataset
        
        # Similarly for the second dataset
        dataset2 = f['/scan/instrument/mlaser/transformations/trans']
        data2 = dataset2[...]  # Load the actual data
        print('Data2 size ',{data2.size}, 'data2 shape ', data2.shape)
        print(f"Dataset type: {type(dataset2)}")  # This prints the type of the h5py dataset
        print(f"Data type: {data2.dtype}")  # This prints the numpy dtype of the dataset's data
        print(f"Attributes: {dataset2.attrs}")  # Attributes of the dataset
        
        
        #dataset3=f['/scan/apd/data']
        print(isinstance(f['/scan/instrument/collection/exp_t01'], h5py.Dataset))  # Should return True if it's a dataset
        dataset3=f['/scan/instrument/collection/exp_t01']
        data3 = dataset3[...]
        print(data3)
        print('Shape: ', data3.shape)
        print('Size ',data3.size)
        print(type(data3))
    
    print(30*'-')


Data size  {21} data shape  (21,)
Dataset type: <class 'h5py._hl.dataset.Dataset'>
Data type: float64
Attributes: <Attributes of HDF5 object at 4872593520>
Data2 size  {1} data2 shape  (1,)
Dataset type: <class 'h5py._hl.dataset.Dataset'>
Data type: float64
Attributes: <Attributes of HDF5 object at 4872593440>
True
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Shape:  (21,)
Size  21
<class 'numpy.ndarray'>
------------------------------


In [13]:
import polars as pl

dlz= pl.LazyFrame({
    "a": [lambda: "a", lambda: "c"]
              }).select(pl.col("a").map_elements(lambda function: function())
                        ) #.collect()
              
# Explain the lazy plan
print('Here')
print(dlz.explain(optimized=True))

# Now collect the results
df = dlz.collect()
print(df)

Here
 SELECT [col("a").map_list()] FROM
  DF ["a"]; PROJECT["a"] 1/1 COLUMNS
shape: (2, 1)
┌─────┐
│ a   │
│ --- │
│ str │
╞═════╡
│ a   │
│ c   │
└─────┘


/var/folders/f5/dfxnk99n5jz64b0dsm0lxfd00000gp/T/ipykernel_3819/2694821941.py:13: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = dlz.collect()


In [14]:
import polars as pl
from pathlib import Path
import h5py
from dataclasses import dataclass



@dataclass
class LazyDatasetReference:
    directory: Path
    file_name: str
    dataset_name: str

    def load_on_demand(self):
        """Load dataset from HDF5 file lazily."""
        file_path = self.directory / self.file_name
        with h5py.File(file_path, "r") as f:
            data = f[self.dataset_name][:]
            
            if data.size == 0:
                return None  # Handle empty datasets
            
            if data.ndim == 1:
                return data.tolist()  # Store as Polars List
            
            if data.ndim == 2:
                return data  # Store as Polars Array

            raise ValueError(f"Unexpected shape {data.shape} in dataset {self.dataset_name}")


# --- 1. Create LazyFrame storing metadata only ---
df = pl.LazyFrame({
    "directory": ["/Users/lotzegud/P08/test_folder2"] * 5,  # Directory path repeated for each file
    "file_name": [
        "h2o_2024_10_16_01043.nxs",
        "h2o_2024_10_16_01044.nxs",
        "h2o_2024_10_16_01045.nxs",
        "h2o_2024_10_16_01046.nxs",
        "h2o_2024_10_16_01047.nxs",
    ],
    "dataset_name": ["/scan/instrument/collection/exp_t01"] * 5,  # Same dataset for all
})

print(df.explain(optimized=True))

# --- 2. Use map_elements to load data lazily ---
df = df.with_columns(
    pl.struct(["directory", "file_name", "dataset_name"])
    .map_elements(lambda row: LazyDatasetReference(
        directory=Path(row["directory"]),
        file_name=row["file_name"],
        dataset_name=row["dataset_name"]
    ).load_on_demand(), return_dtype=pl.Object)  # Adjust dtype as needed
    .alias("data")
)
# --- 3. Collect the results (forcing evaluation) ---
result = df.collect()
print(result)



df = df.with_columns(
    pl.struct(["directory", "file_name", "dataset_name"])
    .map_elements(lambda row: LazyDatasetReference(
        directory=Path(row["directory"]),
        file_name=row["file_name"],
        dataset_name=row["dataset_name"]
    ), return_dtype=pl.Object)
    .alias("data_ref")  # Store reference, NOT loaded data
)
df = df.with_columns(
    pl.col("data_ref").map_elements(lambda ref: ref.load_on_demand(), return_dtype=pl.Object)
    .alias("data2")
)

print(type(df))

print(df.collect())






DF ["directory", "file_name", "dataset_name"]; PROJECT */3 COLUMNS
shape: (5, 4)
┌────────────────────────┬────────────────────────┬────────────────────────┬───────────────────────┐
│ directory              ┆ file_name              ┆ dataset_name           ┆ data                  │
│ ---                    ┆ ---                    ┆ ---                    ┆ ---                   │
│ str                    ┆ str                    ┆ str                    ┆ object                │
╞════════════════════════╪════════════════════════╪════════════════════════╪═══════════════════════╡
│ /Users/lotzegud/P08/te ┆ h2o_2024_10_16_01043.n ┆ /scan/instrument/colle ┆ [1.0, 1.0, 1.0, 1.0,  │
│ st_folde…              ┆ xs                     ┆ ction/ex…              ┆ 1.0, 1.0,…            │
│ /Users/lotzegud/P08/te ┆ h2o_2024_10_16_01044.n ┆ /scan/instrument/colle ┆ [1.0, 1.0, 1.0, 1.0,  │
│ st_folde…              ┆ xs                     ┆ ction/ex…              ┆ 1.0, 1.0,…            │
│ /Users/l

In [ ]:
# Create a dummy LazyFrame
dflazy = pl.LazyFrame({
    "col1": [1, 2, 3],
    "col2": [4, 5, 6]
})

# Explain the logical plan (default, unoptimized)
print(dflazy.explain())

# Explain the optimized logical plan
print(dflazy.explain(optimized=True))

DF ["col1", "col2"]; PROJECT */2 COLUMNS
DF ["col1", "col2"]; PROJECT */2 COLUMNS


In [16]:
from datetime import datetime, timezone

epoch_time = 1729406234.174  # Example epoch timestamp

# Convert to UTC
dt_utc = datetime.fromtimestamp(epoch_time, tz=timezone.utc)
print("UTC Time:", dt_utc.isoformat())

# Convert to local timezone (example: Europe/Berlin, CEST)
import pytz
local_tz = pytz.timezone("Europe/Berlin")
dt_local = dt_utc.astimezone(local_tz)
print("Local Time (CEST):", dt_local.isoformat())


UTC Time: 2024-10-20T06:37:14.174000+00:00
Local Time (CEST): 2024-10-20T08:37:14.174000+02:00
